In [50]:
import numpy as np
from varname import nameof

# adapted from stack overflow
def bmatrix(a, var_name):
    """Stores in a file a LaTeX bmatrix from a numpy array."""
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join([f"{float(x):.6g}" for x in l.split()]) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    with open(f"./docs/aux-matrices/{var_name}.tex", 'w+') as f:
      f.write('\n'.join(rv))

# 1.

In [51]:
# Initial data

x_1 = np.array([[1], [2]])
x_2 = np.array([[-1], [1]])
x_3 = np.array([[1], [0]])

mu_1 = np.array([[2], [2]])
mu_2 = np.array([[0], [0]])

Sigma_1 = np.array([[2, 1], [1, 2]])
Sigma_2 = np.array([[2, 0], [0, 2]])

bmatrix(x_1, nameof(x_1))
bmatrix(x_2, nameof(x_2))
bmatrix(x_3, nameof(x_3))

bmatrix(mu_1, nameof(mu_1))
bmatrix(mu_2, nameof(mu_2))

bmatrix(Sigma_1, nameof(Sigma_1))
bmatrix(Sigma_2, nameof(Sigma_2))

In [52]:
from scipy.stats import multivariate_normal

def calc_likelihoods(x, mu_1, mu_2, Sigma_1, Sigma_2):
  likelihood_1 = multivariate_normal(mu_1, Sigma_1).pdf(x.T)
  likelihood_2 = multivariate_normal(mu_2, Sigma_2).pdf(x.T)
  return np.array([likelihood_1, likelihood_2])

def calc_posteriors(priors, likelihoods):
  posteriors = np.array([])
  for i in range(len(priors)):
    posteriors = np.append(posteriors, priors[i] * likelihoods[i])

  return posteriors / np.sum(posteriors) # normalize

def update_means(k1_posteriors, k2_posteriors):
  mu_1 = np.zeros((2, 1), dtype=float)
  mu_2 = np.zeros((2, 1), dtype=float)

  for i in range(len(k1_posteriors)):
    x = eval(f'x_{i+1}')
    mu_1 += k1_posteriors[i] * x
    mu_2 += k2_posteriors[i] * x

  return mu_1 / np.sum(k1_posteriors), mu_2 / np.sum(k2_posteriors)

def update_covs(k1_posteriors, k2_posteriors, mu_1, mu_2):
  Sigma_1 = np.zeros((2, 2), dtype=float)
  Sigma_2 = np.zeros((2, 2), dtype=float)

  for i in range(len(k1_posteriors)):
    x = eval(f'x_{i+1}')
    Sigma_1 += k1_posteriors[i] * (x - mu_1) @ (x - mu_1).T
    Sigma_2 += k2_posteriors[i] * (x - mu_2) @ (x - mu_2).T

  return Sigma_1 / np.sum(k1_posteriors), Sigma_2 / np.sum(k2_posteriors)

def update_priors(k1_posteriors, k2_posteriors):
  total = np.sum(k1_posteriors) + np.sum(k2_posteriors)
  return np.sum(k1_posteriors) / total, np.sum(k2_posteriors) / total

In [53]:
# because scipy is icky
mu_1_vector = mu_1.transpose()[0]
mu_2_vector = mu_2.transpose()[0]

priors = np.array([0.5, 0.5])

p_x_1_given_k_1, p_x_1_given_k_2 = calc_likelihoods(x_1, mu_1_vector, mu_2_vector, Sigma_1, Sigma_2)
p_x_2_given_k_1, p_x_2_given_k_2 = calc_likelihoods(x_2, mu_1_vector, mu_2_vector, Sigma_1, Sigma_2)
p_x_3_given_k_1, p_x_3_given_k_2 = calc_likelihoods(x_3, mu_1_vector, mu_2_vector, Sigma_1, Sigma_2)

for i in range(1, 4):
  for j in range(1, 3):
    print(f"p(x_{i} | C = k_{j}) = {eval(f'p_x_{i}_given_k_{j}'):0.6g}")

p(x_1 | C = k_1) = 0.0658407
p(x_1 | C = k_2) = 0.0227993
p(x_2 | C = k_1) = 0.00891057
p(x_2 | C = k_2) = 0.0482662
p(x_3 | C = k_1) = 0.0338038
p(x_3 | C = k_2) = 0.061975


In [54]:
posteriors_x_1 = calc_posteriors(priors, [p_x_1_given_k_1, p_x_1_given_k_2])
posteriors_x_2 = calc_posteriors(priors, [p_x_2_given_k_1, p_x_2_given_k_2])
posteriors_x_3 = calc_posteriors(priors, [p_x_3_given_k_1, p_x_3_given_k_2])

for i in range(1, 4):
  for j in range(1, 3):
    print(f"p(C = k_{j} | x_{i}) = {eval(f'posteriors_x_{i}')[j - 1]:0.6g}")

k1_posteriors = np.array([posteriors_x_1[0], posteriors_x_2[0], posteriors_x_3[0]])
k2_posteriors = np.array([posteriors_x_1[1], posteriors_x_2[1], posteriors_x_3[1]])

p(C = k_1 | x_1) = 0.742788
p(C = k_2 | x_1) = 0.257212
p(C = k_1 | x_2) = 0.155843
p(C = k_2 | x_2) = 0.844157
p(C = k_1 | x_3) = 0.352936
p(C = k_2 | x_3) = 0.647064


In [55]:
# update the parameters

mu_1_after_update, mu_2_after_update = update_means(k1_posteriors, k2_posteriors)
Sigma_1_after_update, Sigma_2_after_update = update_covs(k1_posteriors, k2_posteriors, mu_1_after_update, mu_2_after_update)

bmatrix(mu_1_after_update, nameof(mu_1_after_update))
bmatrix(mu_2_after_update, nameof(mu_2_after_update))
bmatrix(Sigma_1_after_update, nameof(Sigma_1_after_update))
bmatrix(Sigma_2_after_update, nameof(Sigma_2_after_update))


In [56]:
# update the priors

priors_after_update = update_priors(k1_posteriors, k2_posteriors)
prior_1_update, prior_2_update = priors_after_update

print(f"p(C = k_1) = {prior_1_update:0.6g}")
print(f"p(C = k_2) = {prior_2_update:0.6g}")

p(C = k_1) = 0.417189
p(C = k_2) = 0.582811


# 2.

### a)

In [57]:
# Given the updated parameters computed in previous question,
# Perform a hard assignment of observations to clusters under a MAP assumption.

mu_1_after_update_vector = mu_1_after_update.transpose()[0]
mu_2_after_update_vector = mu_2_after_update.transpose()[0]

updated_p_x_1_given_k_1, updated_p_x_1_given_k_2 = calc_likelihoods(x_1, mu_1_after_update_vector, mu_2_after_update_vector, Sigma_1_after_update, Sigma_2_after_update)
updated_p_x_2_given_k_1, updated_p_x_2_given_k_2 = calc_likelihoods(x_2, mu_1_after_update_vector, mu_2_after_update_vector, Sigma_1_after_update, Sigma_2_after_update)
updated_p_x_3_given_k_1, updated_p_x_3_given_k_2 = calc_likelihoods(x_3, mu_1_after_update_vector, mu_2_after_update_vector, Sigma_1_after_update, Sigma_2_after_update)

updated_posteriors_x_1 = calc_posteriors(priors_after_update, [updated_p_x_1_given_k_1, updated_p_x_1_given_k_2])
updated_posteriors_x_2 = calc_posteriors(priors_after_update, [updated_p_x_2_given_k_1, updated_p_x_2_given_k_2])
updated_posteriors_x_3 = calc_posteriors(priors_after_update, [updated_p_x_3_given_k_1, updated_p_x_3_given_k_2])

for i in range(1, 4):
  for j in range(1, 3):
    print(f"p(x_{i} | C = k_{j}) = {eval(f'updated_p_x_{i}_given_k_{j}'):0.5g}")
    print(f"p(C = k_{j} | x_{i}) = {eval(f'updated_posteriors_x_{i}')[j - 1]:0.5g}")

# print the hard assignments

hard_assignments = np.argmax(np.array([updated_posteriors_x_1, updated_posteriors_x_2, updated_posteriors_x_3]), axis=1) + 1
print(f"Hard assignments: {hard_assignments}")

p(x_1 | C = k_1) = 0.1957
p(C = k_1 | x_1) = 0.91198
p(x_1 | C = k_2) = 0.01352
p(C = k_2 | x_1) = 0.088017
p(x_2 | C = k_1) = 0.0081953
p(C = k_1 | x_2) = 0.039237
p(x_2 | C = k_2) = 0.14365
p(C = k_2 | x_2) = 0.96076
p(x_3 | C = k_1) = 0.077166
p(C = k_1 | x_3) = 0.34519
p(x_3 | C = k_2) = 0.10478
p(C = k_2 | x_3) = 0.65481
Hard assignments: [1 2 2]


### b)

In [58]:
# Calculate the norm between x1 and x2, x2 and x3, x1 and x3

norm_x1_x2 = np.linalg.norm(x_1 - x_2)
norm_x2_x3 = np.linalg.norm(x_2 - x_3)
norm_x1_x3 = np.linalg.norm(x_1 - x_3)

print(f"norm(x1, x2) = {norm_x1_x2:0.5g}")
print(f"norm(x2, x3) = {norm_x2_x3:0.5g}")
print(f"norm(x1, x3) = {norm_x1_x3:0.5g}")

norm(x1, x2) = 2.2361
norm(x2, x3) = 2.2361
norm(x1, x3) = 2
